In [14]:
import numpy as np
import matplotlib.pyplot as plt
import sys
from sklearn.model_selection import train_test_split

In [15]:
train_data0 = np.load('data0.npy')
train_lab0 = np.load('lab0.npy')

train_data1 = np.load('data1.npy')
train_lab1 = np.load('lab1.npy')

train_data2 = np.load('data2.npy')
train_lab2 = np.load('lab2.npy')

final_train = np.concatenate((train_data0 , train_data1 , train_data2) , axis=0)
final_labels = np.concatenate((train_lab0,train_lab1,train_lab2), axis=0)




X_train, X_val, y_train, y_val = train_test_split(final_train, final_labels, test_size=0.2, random_state=42)

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

In [17]:
class MyDataset(Dataset):
    def __init__(self, images, labels):
        self.images = torch.FloatTensor(images.reshape(-1, 1, 40, 168)) / 255.0
        self.labels = torch.FloatTensor(labels)

    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]

In [18]:
train_dataset = MyDataset(X_train, y_train)
val_dataset = MyDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [19]:
class BaselineCNN(nn.Module):
    def __init__(self):
        super(BaselineCNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, kernel_size=3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
        )
        self.flatten_size = self._get_flatten_size()

        self.fc_layers = nn.Sequential(
            nn.Linear(self.flatten_size, 128),  # have to check for dimensions from conv layers
            nn.ReLU(),
            
            nn.Linear(128, 1)
        )


    

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layers(x)
        return x
    
    def _get_flatten_size(self):
        
        x = torch.randn(1, 1, 40, 168)
        x = self.conv_layers(x)
        return x.numel()

In [20]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BaselineCNN()
model.load_state_dict(torch.load('mymodel.pth', weights_only=True))
model.to(device)

BaselineCNN(
  (conv_layers): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc_layers): Sequential(
    (0): Linear(in_features=26880, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=1, bias=True)
  )
)

In [21]:
model.eval()
example_images, actual_labels = next(iter(val_loader))  
example_images = example_images.to(device) 


predictions = model(example_images).detach().cpu().numpy()
rounded_pred = predictions.round()


for i in range(10): 
    print(f"Prediction: {rounded_pred[i][0]:.2f}, Actual: {actual_labels[i].item():.2f}")


Prediction: 15.00, Actual: 15.00
Prediction: 17.00, Actual: 18.00
Prediction: 17.00, Actual: 19.00
Prediction: 23.00, Actual: 26.00
Prediction: 25.00, Actual: 25.00
Prediction: 9.00, Actual: 14.00
Prediction: 12.00, Actual: 10.00
Prediction: 23.00, Actual: 23.00
Prediction: 26.00, Actual: 19.00
Prediction: 23.00, Actual: 20.00


In [22]:

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, mean_squared_error

def evaluate_model(model, dataloader, device):
    """
    Evaluate model performance using multiple metrics
    """
    model.eval()
    all_predictions = []
    all_labels = []
    total_loss = 0
    criterion = torch.nn.MSELoss()

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            
            
            loss = criterion(outputs.squeeze(), labels)
            total_loss += loss.item() * images.size(0)
            
            
            predictions = outputs.squeeze().cpu().numpy().round()
            all_predictions.extend(predictions)
            all_labels.extend(labels.cpu().numpy())

    
    all_predictions = np.array(all_predictions)
    all_labels = np.array(all_labels)
    
    
    avg_mse = total_loss / len(dataloader.dataset)
    rmse = np.sqrt(avg_mse)

   
    mae = np.mean(np.abs(all_predictions - all_labels))
    
    
    accuracy = np.mean(all_predictions == all_labels)
    
    
    within_one_accuracy = np.mean(np.abs(all_predictions - all_labels) <= 1)

    metrics = {
        "mse": avg_mse,
        "rmse": rmse,
        "mae": mae,
        "exact_accuracy": accuracy * 100,
        "within_one_accuracy": within_one_accuracy * 100
    }
    
    print(metrics)

In [23]:
evaluate_model(model,train_loader,device)

{'mse': 0.5587150933742523, 'rmse': 0.7474724699774917, 'mae': 0.5359167, 'exact_accuracy': 51.35833333333333, 'within_one_accuracy': 95.37916666666668}


In [24]:
evaluate_model(model , val_loader , device)

{'mse': 12.220009432474772, 'rmse': 3.495713007738875, 'mae': 2.7468333, 'exact_accuracy': 12.0, 'within_one_accuracy': 34.46666666666667}
